# Content Recommander

With the content-based filtering we will (based on one movie choice) recommends movies by finding similar movies based on the movie genre, overview and crew

In [1]:
import modules.recommmender.ContentRecommender as cr

#### Getting data
We are going to use data about movies and credits. We load our datasets from our folder "data".

In [2]:
movie_df = cr.load_movies()
movie_df.head()

,Unnamed: 0,Movie_id,title,Genres,release_date,Keywords,overview,poster_path,Budget,Revenue,popularity,vote_average,vote_count
0,0,238,The Godfather,"[{'id': 18, 'name': 'Drama'}, {'id': 80, 'name...",1972-03-14,"[{'id': 131, 'name': 'italy'}, {'id': 697, 'na...","Spanning the years 1945 to 1955, a chronicle o...",/3bhkrj58Vtu7enYsRolD1fZdja1.jpg,245066411,245066411,93.552,8.7,16814
1,1,278,The Shawshank Redemption,"[{'id': 18, 'name': 'Drama'}, {'id': 80, 'name...",1994-09-23,"[{'id': 378, 'name': 'prison'}, {'id': 417, 'n...",Framed in the 1940s for the double murder of h...,/q6y0Go1tsGEsmtFryDOJo3dEmqu.jpg,28341469,28341469,78.664,8.7,22542
2,2,240,The Godfather Part II,"[{'id': 18, 'name': 'Drama'}, {'id': 80, 'name...",1974-12-20,"[{'id': 131, 'name': 'italy'}, {'id': 700, 'na...",In the continuing saga of the Corleone crime f...,/hek3koDUyRQk7FIhPXsa6mT2Zc3.jpg,102600000,102600000,55.752,8.6,10187
3,3,19404,Dilwale Dulhania Le Jayenge,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",1995-10-19,[],"Raj is a rich, carefree, happy-go-lucky second...",/2CAL2433ZeIihfX1Hb2139CX0pW.jpg,100000000,100000000,22.150,8.6,3927
4,4,424,Schindler's List,"[{'id': 18, 'name': 'Drama'}, {'id': 36, 'name...",1993-12-15,"[{'id': 818, 'name': 'based on novel or book'}...",The true story of how businessman Oskar Schind...,/sF1U4EUQS8YHUYjNl3pMGNIQyr0.jpg,321365567,321365567,53.542,8.6,13382


In [3]:
movie_df.shape

(9980, 13)

In [4]:
credit_df = cr.load_credits()
credit_df.head()

,Unnamed: 0,Movie_id,title,Cast,Crew
0,0,238,The Godfather,"[{'adult': False, 'gender': 2, 'id': 3084, 'kn...","[{'adult': False, 'gender': 2, 'id': 154, 'kno..."
1,1,278,The Shawshank Redemption,"[{'adult': False, 'gender': 2, 'id': 504, 'kno...","[{'adult': False, 'gender': 2, 'id': 153, 'kno..."
2,2,240,The Godfather Part II,"[{'adult': False, 'gender': 2, 'id': 1158, 'kn...","[{'adult': False, 'gender': 2, 'id': 154, 'kno..."
3,3,19404,Dilwale Dulhania Le Jayenge,"[{'adult': False, 'gender': 2, 'id': 35742, 'k...","[{'adult': False, 'gender': 1, 'id': 8311, 'kn..."
4,4,424,Schindler's List,"[{'adult': False, 'gender': 2, 'id': 3896, 'kn...","[{'adult': False, 'gender': 2, 'id': 491, 'kno..."


In [5]:
credit_df.shape

(9980, 5)

#### Preparing
We will now clean and combine the two dataframes into one

In [6]:
# prepare credit dataframe (drop title column)
credit_df = cr.prepare_credits(credit_df)
credit_df.head()

,Unnamed: 0,Movie_id,Cast,Crew
0,0,238,"[{'adult': False, 'gender': 2, 'id': 3084, 'kn...","[{'adult': False, 'gender': 2, 'id': 154, 'kno..."
1,1,278,"[{'adult': False, 'gender': 2, 'id': 504, 'kno...","[{'adult': False, 'gender': 2, 'id': 153, 'kno..."
2,2,240,"[{'adult': False, 'gender': 2, 'id': 1158, 'kn...","[{'adult': False, 'gender': 2, 'id': 154, 'kno..."
3,3,19404,"[{'adult': False, 'gender': 2, 'id': 35742, 'k...","[{'adult': False, 'gender': 1, 'id': 8311, 'kn..."
4,4,424,"[{'adult': False, 'gender': 2, 'id': 3896, 'kn...","[{'adult': False, 'gender': 2, 'id': 491, 'kno..."


In [7]:
credit_df.shape

(9980, 4)

In [8]:
# prepare movie dataframe

#### Modeling

In [9]:
# choose movie and size of recommendation
movie_title = "Harry Potter and the Goblet of Fire"
recommends_size = 5

In [14]:
movies = cr.prepare_movies()
movies.head()

,Movie_id,title,tags
0,238,The Godfather,drama crime italy lossoflovedone loveatfirstsi...
1,278,The Shawshank Redemption,drama crime prison corruption policebrutality ...
2,240,The Godfather Part II,drama crime italy italianamerican cuba symboli...
3,19404,Dilwale Dulhania Le Jayenge,comedy drama romance shahrukhkhan kajol amrish...
4,424,Schindler's List,drama history war basedonnovelorbook factory c...


In [13]:
# vectors
vector = cr.create_matrix(movies)
# vector
similarity = cr.find_similarities(vector)
similarity

# get movie index

# distance

# get the recommendations

array([[1.        , 0.15118579, 0.43826153, ..., 0.06482037, 0.01801875,
        0.14580296],
       [0.15118579, 1.        , 0.08834522, ..., 0.03429972, 0.0381385 ,
        0.        ],
       [0.43826153, 0.08834522, 1.        , ..., 0.1136331 , 0.        ,
        0.13631964],
       ...,
       [0.06482037, 0.03429972, 0.1136331 , ..., 1.        , 0.0327035 ,
        0.13231403],
       [0.01801875, 0.0381385 , 0.        , ..., 0.0327035 , 1.        ,
        0.        ],
       [0.14580296, 0.        , 0.13631964, ..., 0.13231403, 0.        ,
        1.        ]])

#### Result

In [11]:
cr.recommend_movies(movie_title, recommends_size)

['Harry Potter and the Deathly Hallows: Part 2',
 'Harry Potter and the Prisoner of Azkaban',
 "Harry Potter and the Philosopher's Stone",
 'Harry Potter and the Order of the Phoenix',
 'Harry Potter and the Chamber of Secrets']